In [1]:
%pylab inline
import os
import trako as TKO

Populating the interactive namespace from numpy and matplotlib


In [2]:
DATASETNAME = 'WholeBrain'
DATADIR = '/home/d/DATA/Dropbox/'+DATASETNAME+'/'
SUBJECTS = ['']
GROUP = ['']

# Trako

In [3]:
OUTPUTDIR = '/home/d/DATA/Dropbox/'+DATASETNAME+'_TKO/'
vtpsizes = []
tkosizes = []
for S in SUBJECTS:
    for G in GROUP:
        CURRENTDIR = os.path.join(DATADIR, S, G)
        CURRENTOUTDIR = os.path.join(OUTPUTDIR, S, G)
        files = os.listdir(CURRENTDIR)
        
        if not os.path.exists(CURRENTOUTDIR):
            os.makedirs(CURRENTOUTDIR)
        
        for f in files:
            if not f.endswith('vtk'):
                continue
            
            vtpfile = os.path.join(CURRENTDIR, f)
            vtpsize = os.path.getsize(vtpfile)
            tkofile = os.path.join(CURRENTOUTDIR, f.replace('vtk', 'tko'))
            
            if not os.path.exists(tkofile):
                tkoed = TKO.Encoder.fromVtp(vtpfile, verbose=False)
                tkoed.save(tkofile)
                
            tkosize = os.path.getsize(tkofile)
            print('Converted', f, '- Ratio:', float(vtpsize)/tkosize)
            vtpsizes.append(vtpsize)
            tkosizes.append(tkosize)
            

Converted RealTractDataIncludingTensorsAndScalars.vtk - Ratio: 2.0295713967948203


In [4]:
ratios = []
for i,v in enumerate(vtpsizes):
    ratios.append(vtpsizes[i]/float(tkosizes[i]))

# VTP GZip

In [6]:
import gzip, shutil
OUTPUTDIR = '/home/d/DATA/Dropbox/'+DATASETNAME+'_GZ/'
gzsizes = []
for S in SUBJECTS:
    for G in GROUP:
        CURRENTDIR = os.path.join(DATADIR, S, G)
        CURRENTOUTDIR = os.path.join(OUTPUTDIR, S, G)
        files = os.listdir(CURRENTDIR)
        
        if not os.path.exists(CURRENTOUTDIR):
            os.makedirs(CURRENTOUTDIR)
        
        for f in files:
            if not f.endswith('vtk'):
                continue
            
            vtpfile = os.path.join(CURRENTDIR, f)
            vtpsize = os.path.getsize(vtpfile)
            gzfile = os.path.join(CURRENTOUTDIR, f.replace('vtk', 'vtk.gz'))
            
            if not os.path.exists(gzfile):
                
                with open(vtpfile, 'rb') as f_in:
                    with gzip.open(gzfile, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                        
            gzsize = os.path.getsize(gzfile)
            print('Converted', f, '- Ratio:', float(vtpsize)/gzsize)
            gzsizes.append(gzsize)
            

Converted RealTractDataIncludingTensorsAndScalars.vtk - Ratio: 1.3708939347911664


In [7]:
gz_ratios = []
for i,v in enumerate(vtpsizes):
    gz_ratios.append(vtpsizes[i]/float(gzsizes[i]))

# Plots

In [8]:
def reject_outliers(data, m=2):
    data = np.array(data)
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [9]:
import plotly.graph_objects as go

In [14]:
colors = ['lightslategray']*3
colors[2] = 'royalblue'

fig = go.Figure(data=[go.Bar(
    x=['VTP', 'VTP (GZip)', 
#        'TRK',
#        'TRK (GZip)', 
       'Trako'],
    y=[np.sum(vtpsizes)/1000000., 
       np.sum(gzsizes)/1000000.,
#        np.sum(trksizes)/1000000.,
#        np.sum(trkgzsizes)/1000000.,
       np.sum(tkosizes)/1000000.,
      ],
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(yaxis_title="Megabytes")
fig.show()


In [22]:
colors = ['lightslategray',] * 3
colors[2] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=['VTP', 'VTP (GZip)', 
#        'TRK',
#        'TRK (GZip)', 
       'Trako'],
    y=[1, 
       gz_ratios[0],
#        np.mean(reject_outliers(trk_ratios)),
#        np.mean(reject_outliers(np.array(trkgz_ratios)<20)),
       ratios[0],
      ],
#     error_y=dict(type='data', array=[0, 
#        np.std(reject_outliers(gz_ratios)),
# #        np.std(reject_outliers(trk_ratios)),
# #        np.std(reject_outliers(np.array(trkgz_ratios)<20)),
#        np.std(reject_outliers(ratios)),
#       ]),
    marker_color=colors # marker color can be a single color value or an iterable
)])
fig.update_layout(yaxis_title="Compression factor")

fig.show()
